In [1]:
import pandas as pd
from scipy import stats
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import unicodedata
from __future__ import annotations

In [2]:
allstars = pd.read_table('all_stars_drama_raw.tsv')

In [3]:
allstars

,id,label,isGroup,gender,text,corpus,playName
0,mirvolin,Мирволин,False,MALE,"Здравствуй, Герасим! как поживаешь?.. А что, о...",rus,turgenev-zavtrak-u-predvoditelja
1,gerasim,Герасим,False,MALE,Здравствуйте. Где это вы достали лошадь?\nКто ...,rus,turgenev-zavtrak-u-predvoditelja
2,balagalaev,Балагалаев,False,MALE,"Филька! Вельвицкого мне позвать.\nТак, так, та...",rus,turgenev-zavtrak-u-predvoditelja
3,velvitskij,Вельвицкий,False,MALE,"Как же-с, как же-с.\nСлушаю-с. Так, стало быть...",rus,turgenev-zavtrak-u-predvoditelja
4,alupkin,Алупкин,False,MALE,"Доложи. А-а-лупкин, дворянин…\nПокорнейше благ...",rus,turgenev-zavtrak-u-predvoditelja
...,...,...,...,...,...,...,...
34481,SAILORS.TYRE.1_Per,Sailor,False,MALE,Where is Lord Helicanus ? He can resolve you ....,shake,pericles
34482,GENTLEMEN.TYRE.0.1_Per,Gentleman,False,MALE,Doth your Lordship call ?,shake,pericles
34483,SAILORS.MYTILENE.1_Per,Sailor,False,MALE,"Sir ,\nThis is the man that can , in aught you...",shake,pericles
34484,LORDS.MYTILENE.0.1_Per,Lord,False,MALE,"Sir , we have a maid in Mytilene ,\nI durst wa...",shake,pericles


In [22]:
allstars[allstars['text'].apply(lambda x: not isinstance(x, str))]

,id,label,isGroup,gender,text,corpus,playName
10968,le-docteur-sans-pareil,Le Docteur sans Pareil,False,UNKNOWN,NaN,fre,hervilly-docteur-sans-pareil
19609,der_klavierspieler,Der Klavierspieler,False,MALE,NaN,ger,schnitzler-zum-grossen-wurstel
19860,die_anderen_1-1,Die Anderen,True,MALE,NaN,ger,anzengruber-heimgfunden
19965,ein_knappe,Ein Knappe,False,MALE,NaN,ger,hebbel-agnes-bernauer
19975,schildknapp,Ein Schildknapp,False,MALE,NaN,ger,hebbel-agnes-bernauer
...,...,...,...,...,...,...,...
32779,dritter_herr,3. Herr,False,MALE,NaN,ger,chlumberg-die-fuehrer
32921,zehnte_gruppe,Zehnte Gruppe,True,UNKNOWN,NaN,ger,chlumberg-wunder-um-verdun
33030,die_magd,Die Magd,False,FEMALE,NaN,ger,bahr-das-konzert
33548,Epilogue_2H4,Epilogue,False,UNKNOWN,NaN,shake,henry-iv-part-ii


In [29]:
# drop all rows where the text isn't a string

allstars.drop(allstars[allstars['text'].apply(lambda x: not isinstance(x, str))].index, inplace=True)

In [30]:
allstars

,id,label,isGroup,gender,text,corpus,playName
0,mirvolin,Мирволин,False,MALE,"Здравствуй, Герасим! как поживаешь?.. А что, о...",rus,turgenev-zavtrak-u-predvoditelja
1,gerasim,Герасим,False,MALE,Здравствуйте. Где это вы достали лошадь?\nКто ...,rus,turgenev-zavtrak-u-predvoditelja
2,balagalaev,Балагалаев,False,MALE,"Филька! Вельвицкого мне позвать.\nТак, так, та...",rus,turgenev-zavtrak-u-predvoditelja
3,velvitskij,Вельвицкий,False,MALE,"Как же-с, как же-с.\nСлушаю-с. Так, стало быть...",rus,turgenev-zavtrak-u-predvoditelja
4,alupkin,Алупкин,False,MALE,"Доложи. А-а-лупкин, дворянин…\nПокорнейше благ...",rus,turgenev-zavtrak-u-predvoditelja
...,...,...,...,...,...,...,...
34481,SAILORS.TYRE.1_Per,Sailor,False,MALE,Where is Lord Helicanus ? He can resolve you ....,shake,pericles
34482,GENTLEMEN.TYRE.0.1_Per,Gentleman,False,MALE,Doth your Lordship call ?,shake,pericles
34483,SAILORS.MYTILENE.1_Per,Sailor,False,MALE,"Sir ,\nThis is the man that can , in aught you...",shake,pericles
34484,LORDS.MYTILENE.0.1_Per,Lord,False,MALE,"Sir , we have a maid in Mytilene ,\nI durst wa...",shake,pericles


In [77]:
allstars.cleanText.map(lambda x: len(x.split(' '))).describe()

count    34414.000000
mean       665.104754
std       1084.004896
min          1.000000
25%         35.000000
50%        211.000000
75%        843.000000
max      29414.000000
Name: cleanText, dtype: float64

In [42]:
def strip_punctuation(text: str) -> str:
    """
    >>> strip_punctuation(u'something')
    u'something'

    >>> strip_punctuation(u'something.,:else really')
    u'somethingelse really'
    """
    punctutation_cats = set(['Pc', 'Pd', 'Ps', 'Pe', 'Pi', 'Pf', 'Po'])
    return ''.join(x for x in text
                   if unicodedata.category(x) not in punctutation_cats)

In [51]:
def normalise_spaces(text: str) -> str:
    # by default, Python splits on any whitespace eg "a  \t\nb" -> ['a','b']
    return ' '.join(text.split()).strip()

In [58]:
strip_punctuation(normalise_spaces("Je suis      prêt à \t\tt'écouter, mon\n cher frère,")).lower()

'je suis prêt à técouter mon cher frère'

In [68]:
cleanText = allstars.text.map(lambda x: normalise_spaces(strip_punctuation(x).lower()))
allstars['cleanText'] = cleanText
allstars

,id,label,isGroup,gender,text,corpus,playName,cleanText
0,mirvolin,Мирволин,False,MALE,"Здравствуй, Герасим! как поживаешь?.. А что, о...",rus,turgenev-zavtrak-u-predvoditelja,здравствуй герасим как поживаешь а что он еще ...
1,gerasim,Герасим,False,MALE,Здравствуйте. Где это вы достали лошадь?\nКто ...,rus,turgenev-zavtrak-u-predvoditelja,здравствуйте где это вы достали лошадь кто пре...
2,balagalaev,Балагалаев,False,MALE,"Филька! Вельвицкого мне позвать.\nТак, так, та...",rus,turgenev-zavtrak-u-predvoditelja,филька вельвицкого мне позвать так так тактакт...
3,velvitskij,Вельвицкий,False,MALE,"Как же-с, как же-с.\nСлушаю-с. Так, стало быть...",rus,turgenev-zavtrak-u-predvoditelja,как жес как жес слушаюс так стало бытьс бумаги...
4,alupkin,Алупкин,False,MALE,"Доложи. А-а-лупкин, дворянин…\nПокорнейше благ...",rus,turgenev-zavtrak-u-predvoditelja,доложи аалупкин дворянин покорнейше благодарю ...
...,...,...,...,...,...,...,...,...
34481,SAILORS.TYRE.1_Per,Sailor,False,MALE,Where is Lord Helicanus ? He can resolve you ....,shake,pericles,where is lord helicanus he can resolve you o h...
34482,GENTLEMEN.TYRE.0.1_Per,Gentleman,False,MALE,Doth your Lordship call ?,shake,pericles,doth your lordship call
34483,SAILORS.MYTILENE.1_Per,Sailor,False,MALE,"Sir ,\nThis is the man that can , in aught you...",shake,pericles,sir this is the man that can in aught you woul...
34484,LORDS.MYTILENE.0.1_Per,Lord,False,MALE,"Sir , we have a maid in Mytilene ,\nI durst wa...",shake,pericles,sir we have a maid in mytilene i durst wager w...


In [86]:
allstars_long = allstars[allstars.cleanText.apply(lambda x: 2000 < len(x.split(' ')) < 20000)]
allstars_long

,id,label,isGroup,gender,text,corpus,playName,cleanText
2,balagalaev,Балагалаев,False,MALE,"Филька! Вельвицкого мне позвать.\nТак, так, та...",rus,turgenev-zavtrak-u-predvoditelja,филька вельвицкого мне позвать так так тактакт...
21,darja_ivanovna,Дарья Ивановна,False,FEMALE,. Миша!\nВы… ходили к Попову?\nЧто он вам сказ...,rus,turgenev-provintsialka,миша вы ходили к попову что он вам сказал жду ...
27,graf_ljubin,Граф Любин,False,MALE,". Что же это значит? Никого нет, что ли? Зачем...",rus,turgenev-provintsialka,что же это значит никого нет что ли зачем этот...
28,donja_dolores,Донья Долорес,False,FEMALE,". Однако, мне очень скучно. — Мне нечего читат...",rus,turgenev-neostorozhnost,однако мне очень скучно мне нечего читать я не...
29,don_rafael,Дон Рафаэль,False,MALE,". Что бы вы сделали, прекрасная сеньйора?\n. С...",rus,turgenev-neostorozhnost,что бы вы сделали прекрасная сеньйора сеньйора...
...,...,...,...,...,...,...,...,...
34399,Cloten_Cym,Cloten,False,MALE,"If my shirt were bloody , then to shift it . H...",shake,cymbeline,if my shirt were bloody then to shift it have ...
34402,Iachimo_Cym,Iachimo,False,MALE,"Believe it , sir , I have seen him in Britain ...",shake,cymbeline,believe it sir i have seen him in britain he w...
34410,Belarius_Cym,Belarius,False,MALE,A goodly day not to keep house with such\nWhos...,shake,cymbeline,a goodly day not to keep house with such whose...
34432,Gower_Per,Gower,False,MALE,"To sing a song that old was sung ,\nFrom ashes...",shake,pericles,to sing a song that old was sung from ashes an...


In [88]:
allstars.drop(['text'],axis=1).to_pickle('allstars_clean.pkl.zip')

In [89]:
allstars_long.drop(['text'],axis=1).to_pickle('allstars_long.pkl.zip')

In [87]:
allstars_long

,id,label,isGroup,gender,text,corpus,playName,cleanText
2,balagalaev,Балагалаев,False,MALE,"Филька! Вельвицкого мне позвать.\nТак, так, та...",rus,turgenev-zavtrak-u-predvoditelja,филька вельвицкого мне позвать так так тактакт...
21,darja_ivanovna,Дарья Ивановна,False,FEMALE,. Миша!\nВы… ходили к Попову?\nЧто он вам сказ...,rus,turgenev-provintsialka,миша вы ходили к попову что он вам сказал жду ...
27,graf_ljubin,Граф Любин,False,MALE,". Что же это значит? Никого нет, что ли? Зачем...",rus,turgenev-provintsialka,что же это значит никого нет что ли зачем этот...
28,donja_dolores,Донья Долорес,False,FEMALE,". Однако, мне очень скучно. — Мне нечего читат...",rus,turgenev-neostorozhnost,однако мне очень скучно мне нечего читать я не...
29,don_rafael,Дон Рафаэль,False,MALE,". Что бы вы сделали, прекрасная сеньйора?\n. С...",rus,turgenev-neostorozhnost,что бы вы сделали прекрасная сеньйора сеньйора...
...,...,...,...,...,...,...,...,...
34399,Cloten_Cym,Cloten,False,MALE,"If my shirt were bloody , then to shift it . H...",shake,cymbeline,if my shirt were bloody then to shift it have ...
34402,Iachimo_Cym,Iachimo,False,MALE,"Believe it , sir , I have seen him in Britain ...",shake,cymbeline,believe it sir i have seen him in britain he w...
34410,Belarius_Cym,Belarius,False,MALE,A goodly day not to keep house with such\nWhos...,shake,cymbeline,a goodly day not to keep house with such whose...
34432,Gower_Per,Gower,False,MALE,"To sing a song that old was sung ,\nFrom ashes...",shake,pericles,to sing a song that old was sung from ashes an...


In [102]:
def threegrams(text: str)->list[str]:
    return [text[i:i+3] for i in range(len(text)-3+1)]

In [104]:
len(threegrams(allstars_long.iloc[-1].cleanText))

23239

In [107]:
len(allstars_long.iloc[-1].cleanText.split(' '))

4580

In [115]:
c1 = Counter(threegrams(allstars_long.iloc[-1].cleanText))

In [122]:
len(c1)

2282

In [227]:
c1.most_common(10)

[(' th', 504),
 ('the', 281),
 ('he ', 178),
 ('nd ', 172),
 (' an', 154),
 (' to', 150),
 ('and', 145),
 ('e t', 142),
 ('to ', 138),
 ('re ', 132)]

In [229]:
c2 = Counter()
_ = allstars_long[allstars_long.corpus=="shake"].cleanText.map(lambda x: c2.update(threegrams(x)))
len(c2)

5921

In [228]:
c2.most_common(10)

[(' th', 42422),
 ('the', 24879),
 ('he ', 17327),
 ('nd ', 16951),
 ('and', 15140),
 (' an', 15044),
 ('is ', 12427),
 (' to', 12096),
 ('ll ', 11969),
 (' yo', 11762)]

In [223]:
pd.Series(c1.values()).describe()

count    2282.000000
mean       10.183611
std        21.328089
min         1.000000
25%         1.000000
50%         4.000000
75%        10.000000
max       504.000000
dtype: float64

In [224]:
pd.Series(c2.values()).describe()

count     5921.000000
mean       364.609525
std       1272.688895
min          1.000000
25%          4.000000
50%         31.000000
75%        207.000000
max      42422.000000
dtype: float64

In [238]:
c3 = Counter(allstars_long[allstars_long.corpus=='shake'].cleanText.apply(threegrams).explode().values)

In [397]:
def threegram_bag(df: pd.Series) -> np.ndarray:
    # need to dropna because some chars still yield 
    # no 3grams (maybe they say 'lo' or something) 
    # and explode turns that into nan
    return df.apply(threegrams).explode().dropna().values

In [262]:
def unique_threegrams(c):
    return len(set(threegram_bag(allstars_long[allstars_long.corpus==c].cleanText)))

In [257]:
def total_threegrams(c):
    return len(threegram_bag(allstars_long[allstars_long.corpus==c].cleanText))

In [263]:
[(c, unique_threegrams(c)) for c in allstars_long.corpus.unique()]

[('rus', 12542), ('fre', 9939), ('ger', 14289), ('shake', 5921)]

In [259]:
[(c, total_threegrams(c)/1_000_000) for c in allstars_long.corpus.unique()]

[('rus', 4.048757),
 ('fre', 29.722629),
 ('ger', 24.409509),
 ('shake', 2.158853)]

In [522]:
# quick POC

def bootstrap(a, b, fn, n=999, alpha=0.05):
    res = np.zeros(n)
    for i in range(n):
        res[i] = fn(a,b)
    res.sort()
    alphas = np.array([alpha/2, 0.25, 0.5, 0.75, 1 - alpha/2])
    nvals = np.round((n - 1) * alphas).astype(int)
    return res[nvals]

df_trim = allstars_long.iloc[-34:]

for pn, play_df in df_trim.groupby('playName'):
    all = allstars[(allstars.playName==pn)]
    all_threegrams = threegram_bag(all.cleanText)

    # We need a vectorizer that takes an existing sequence of 
    # 3-grams to work with the way we're doing sampling.
    cv = CountVectorizer(analyzer=lambda x: x)

    # Now we can transform any vector of 3-grams and the feature 
    # universes will all be the same. fit on the whole play to 
    # learn the full 3-gram vocab
    cv.fit([all_threegrams])
    print('='*40)
    print(f"Starting {pn} -- total 3-gram vocab is {len(cv.get_feature_names_out())}, total cast is {len(all)}")

    # define this here to catch the vectorizer (cv) in its scope
    def wasserstein(a,b):
        # +transform+ returns a 
        # sparse CSR matrix, which we convert to an np.ndarray and 
        # then take the first row. Samples are the same size, so we 
        # don't need to scale. np.random.choice is a sample with
        # replacement (normal for bootstrap samples)
        samp1 = cv.transform([np.random.choice(a,10000)]).toarray()[0]
        samp2 = cv.transform([np.random.choice(b,10000)]).toarray()[0]
        return stats.wasserstein_distance(samp1, samp2)

    for r in play_df.itertuples(index=False):
        others = allstars[(allstars.playName==r.playName) & (r.label != allstars.label)]
        this_char_threegrams = threegrams(r.cleanText)
        other_chars_threegrams = threegram_bag(others.cleanText)

        print((f"Total vocab for "
            f"{r.label} is {len(set(this_char_threegrams))} "
            f"({len(set(this_char_threegrams))/len(cv.get_feature_names_out())*100:.2f}%)"
        ))
        print(f"{len(this_char_threegrams)} threegrams for {r.label} ({(len(this_char_threegrams)/len(other_chars_threegrams))*100:.1f}%)")

        print(f"bootstrap vs other: {bootstrap(this_char_threegrams, other_chars_threegrams, wasserstein)}")
        print(f"bootstrap vs self: {bootstrap(this_char_threegrams, this_char_threegrams, wasserstein)}")
        print('-'*40)



Starting antony-and-cleopatra -- total 3-gram vocab is 3514, total cast is 74
Total vocab for Cleopatra is 2378 (67.67%)
23525 threegrams for Cleopatra (23.9%)
bootstrap vs other: [0.11383039 0.13887308 0.15537849 0.17131474 0.20546386]
bootstrap vs self: [0.06829824 0.08423449 0.09561753 0.10756972 0.13602732]
----------------------------------------
Total vocab for Antony is 2556 (72.74%)
30284 threegrams for Antony (33.1%)
bootstrap vs other: [0.09334092 0.11269209 0.12464428 0.13830393 0.16733068]
bootstrap vs self: [0.06829824 0.08594195 0.09618668 0.10870803 0.13488902]
----------------------------------------
Total vocab for Enobarbus Domitius is 2004 (57.03%)
12593 threegrams for Enobarbus Domitius (11.5%)
bootstrap vs other: [0.13659647 0.1587934  0.17074559 0.18383608 0.21115538]
bootstrap vs self: [0.06886739 0.0865111  0.09789414 0.10984633 0.13716562]
----------------------------------------
Total vocab for Octavius Caesar is 2174 (61.87%)
14937 threegrams for Octavius Cae